# Accès aux interfaces numériques
## Taux de couverture mobile (2G, 3G, 4G ...)

In [9]:
# Donnees de mon reseau mobile: sites mobiles en France métropolitaine au 30 juin 2020
# https://www.data.gouv.fr/fr/datasets/r/77ca5457-c1fe-4450-9761-1a6a598921c0
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

external_data = Path('../data/external/')
processed_data = Path('../data/processed/')
raw_data = Path('../data/raw/')
interim_data = Path('../data/interim/')

In [10]:
df = pd.read_csv(raw_data/'sites_mobiles_2020_juin.csv', delimiter=';', error_bad_lines=False, low_memory=False)
df = df.drop_duplicates()
df

,code_op,nom_op,num_site,x_lambert_93,y_lambert_93,nom_reg,nom_dep,insee_dep,nom_com,insee_com,site_2g,site_3g,site_4g,mes_4g_trim,site_ZB,site_DCC
0,20801,Orange,0012290010,872639,6570768,AUVERGNE RHONE ALPES,AIN,01,BOURG EN BRESSE,01053,1,1,1,0,0,0
1,20801,Orange,0012290011,860279,6529851,AUVERGNE RHONE ALPES,AIN,01,DAGNEUX,01142,1,1,1,0,0,0
2,20801,Orange,0012290012,852231,6532639,AUVERGNE RHONE ALPES,AIN,01,TRAMOYES,01424,1,1,1,0,0,0
3,20801,Orange,0012290014,901020,6564540,AUVERGNE RHONE ALPES,AIN,01,NANTUA,01269,1,1,1,0,0,0
4,20801,Orange,0012290016,903596,6563175,AUVERGNE RHONE ALPES,AIN,01,NEYROLLES,01274,1,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86439,20820,Bouygues Telecom,ZPH84201,908817,6299061,PROVENCE-ALPES-COTE D'AZUR,VAUCLUSE,84,GRAMBOIS,84052,0,1,0,0,1,0
86440,20820,Bouygues Telecom,ZPH88202,907166,6820434,GRAND EST,VOSGES,88,MARTIGNY-LES-GERBONVAUX,88290,0,1,0,0,1,0
86441,20820,Bouygues Telecom,ZPH88203,916863,6766415,GRAND EST,VOSGES,88,GRIGNONCOURT,88220,0,1,1,0,1,0
86442,20820,Bouygues Telecom,ZPH88204,919671,6785054,GRAND EST,VOSGES,88,PROVENCHERES-LES-DARNEY,88360,0,1,1,0,1,0


Pour calculer le taux de couverture mobile, on va suposer que obtenir:
- la 4G correspond à un poids de 50%
- la 3G --> poids de 30%
- la 2G --> poids de 20%

In [11]:
cols = 'nom_op nom_com insee_dep insee_com site_2g site_3g site_4g'.split()
df = df[cols]
df.head()

,nom_op,nom_com,insee_dep,insee_com,site_2g,site_3g,site_4g
0,Orange,BOURG EN BRESSE,01,01053,1,1,1
1,Orange,DAGNEUX,01,01142,1,1,1
2,Orange,TRAMOYES,01,01424,1,1,1
3,Orange,NANTUA,01,01269,1,1,1
4,Orange,NEYROLLES,01,01274,1,1,1


In [12]:
df['nom_op'].value_counts()

Orange              25141
SFR                 22267
Bouygues Telecom    21005
Free Mobile         17918
Name: nom_op, dtype: int64

In [13]:
# Il n'y a que la moitié des communes avec de la donnée
df['insee_com'].unique().shape

(17075,)

In [14]:
# 13 000 communes couvertes par Orange par ex.
df[df['nom_op'] == 'Orange']['insee_com'].unique().shape

(13371,)

In [15]:
df['site_2g'] = df['site_2g'].map({1: 0.2, 0:0})
df['site_3g'] = df['site_3g'].map({1: 0.3, 0:0})
df['site_4g'] = df['site_4g'].map({1: 0.5, 0:0})

df['COUVERTURE_MOBILE'] = df[['site_2g', 'site_3g', 'site_4g']].sum(axis=1)
df = df.rename(columns={'insee_com': 'CODE_INSEE', 'insee_dep': 'DEP'})
df = df.drop(columns=['site_2g', 'site_3g', 'site_4g'])

In [16]:
# COMMUNES

communes = pd.read_csv(raw_data/'table_insee_libcom_dep.csv').drop(columns=['Unnamed: 0'])
communes

,CODE_INSEE,LIBCOM,DEP
0,01001,L'Abergement-Clémenciat,01
1,01002,L'Abergement-de-Varey,01
2,01004,Ambérieu-en-Bugey,01
3,01005,Ambérieux-en-Dombes,01
4,01006,Ambléon,01
...,...,...,...
35005,97613,M'Tsangamouji,976
35006,97614,Ouangani,976
35007,97615,Pamandzi,976
35008,97616,Sada,976


In [17]:
# MERGE des données avec les communes + code INSEE

res = df.merge(communes, on=['CODE_INSEE', 'DEP'], how='inner')
res

,nom_op,nom_com,DEP,CODE_INSEE,COUVERTURE_MOBILE,LIBCOM
0,Orange,BOURG EN BRESSE,01,01053,1.0,Bourg-en-Bresse
1,Orange,BOURG EN BRESSE,01,01053,1.0,Bourg-en-Bresse
2,Orange,BOURG EN BRESSE,01,01053,1.0,Bourg-en-Bresse
3,Orange,BOURG EN BRESSE,01,01053,1.0,Bourg-en-Bresse
4,Orange,BOURG EN BRESSE,01,01053,1.0,Bourg-en-Bresse
...,...,...,...,...,...,...
83885,Bouygues Telecom,MISSEGRE,11,11235,0.5,Missègre
83886,Bouygues Telecom,ALZON,30,30009,0.5,Alzon
83887,Bouygues Telecom,PAILHARES,07,07170,1.0,Pailharès
83888,Bouygues Telecom,GRURY,71,71227,1.0,Grury


In [18]:
couv_mobile_temp = res.groupby('CODE_INSEE')['COUVERTURE_MOBILE'].median()
couv_mobile = pd.DataFrame({'CODE_INSEE': res['CODE_INSEE'].drop_duplicates().values, 
                    'COUVERTURE_MOBILE': couv_mobile_temp.values})

couv_mobile = couv_mobile.merge(communes, on=['CODE_INSEE'], how='right')
temp = couv_mobile['COUVERTURE_MOBILE']
couv_mobile = couv_mobile.drop(columns=['COUVERTURE_MOBILE'])
couv_mobile['COUVERTURE_MOBILE'] = temp
couv_mobile

,CODE_INSEE,LIBCOM,DEP,COUVERTURE_MOBILE
0,01001,L'Abergement-Clémenciat,01,NaN
1,01002,L'Abergement-de-Varey,01,1.00
2,01004,Ambérieu-en-Bugey,01,1.00
3,01005,Ambérieux-en-Dombes,01,1.00
4,01006,Ambléon,01,0.65
...,...,...,...,...
35005,97613,M'Tsangamouji,976,NaN
35006,97614,Ouangani,976,NaN
35007,97615,Pamandzi,976,NaN
35008,97616,Sada,976,NaN


In [19]:
couv_mobile.to_csv(f'{interim_data}/taux_couverture_mobile.csv', index=False)

## DOM -TOM

Cette liste présente, pour chaque site, le code MCC-MNC de l’opérateur (operateur), la longitude (X) et la latitude (Y) du site, et si le site est équipé en 2G (C2G) et/ou en 3G (C3G) et/ou en 4G (C4G).

In [20]:
domtom = pd.read_csv(raw_data/'MAR_sites_mobiles_2020_T2.csv', delimiter=';', error_bad_lines=False, low_memory=False)
domtom

,Operateur,X,Y,C2G,C3G,C4G
0,34020,-60.949451,14.691945,1,1,1
1,34020,-60.981388,14.677198,1,1,1
2,34020,-61.029451,14.778889,1,1,1
3,34020,-61.066673,14.813611,1,1,1
4,34020,-61.106951,14.753889,1,0,0
...,...,...,...,...,...,...
479,34002,-61.066900,14.518100,1,1,1
480,34002,-60.876100,14.602000,0,1,1
481,34002,-60.984100,14.765900,1,1,1
482,34002,-60.995800,14.646600,1,1,1


In [22]:
import geopy

def get_zipcode(df, geolocator, lat_field, lon_field):
    location = geolocator.reverse((df[lat_field], df[lon_field]))
    return location.raw['address']['postcode']

geolocator = geopy.Nominatim(user_agent="geoapiExercises")

zipcodes = domtom.apply(get_zipcode, axis=1, geolocator=geolocator, lat_field='Y', lon_field='X')
zipcodes

0      97231
1      97231
2      97230
3      97214
4      97260
       ...  
479    97217
480    97240
481    97230
482    97232
483    97212
Length: 484, dtype: object